In [ ]:
# 1. Instalamos librerías del sistema para manejo de audio
!apt-get install -y ffmpeg libavcodec-dev libavformat-dev libavdevice-dev libavfilter-dev libswscale-dev libswresample-dev libpostproc-dev

# 2. Instalamos la "Trinidad" de PyTorch en versión compatible (2.5.1)
!pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124

# 3. Instalamos Audiocraft (MusicGen) y dependencias críticas
!pip install av==12.3.0
!pip install --no-deps git+https://github.com/facebookresearch/audiocraft#egg=audiocraft
!pip install demucs encodec flashy>=0.0.1 hydra-core>=1.1 hydra_colorlog julius num2words omegaconf pesq pystoi torchdiffeq torchmetrics xformers==0.0.28.post3
!pip install "numpy<2.0.0"

print("✅ Instalación completada. REINICIA LA SESIÓN (Entorno de ejecución > Reiniciar sesión) antes de seguir.")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following NEW packages will be installed:
  libavcodec-dev libavdevice-dev libavfilter-dev libavformat-dev libavutil-dev
  libpostproc-dev libswresample-dev libswscale-dev
0 upgraded, 8 newly installed, 0 to remove and 41 not upgraded.
Need to get 10.2 MB of archives.
After this operation, 39.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libavutil-dev amd64 7:4.4.2-0ubuntu0.22.04.1 [427 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libswresample-dev amd64 7:4.4.2-0ubuntu0.22.04.1 [78.0 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libavcodec-dev amd64 7:4.4.2-0ubuntu0.22.04.1 [6,221 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libavformat-dev amd64 7:4.4.2-0ubuntu0.22.04.1 [1,347 

✅ Instalación completada. REINICIA LA SESIÓN (Entorno de ejecución > Reiniciar sesión) antes de seguir.


In [ ]:
# Celda 2: Configuración del Modelo Melody 😍
from audiocraft.models import musicgen
from audiocraft.utils.notebook import display_audio
import torch
import gc

# Limpieza de memoria para evitar errores
gc.collect()
torch.cuda.empty_cache()

# Cargamos el modelo MELODY (necesario para usar cancion.mp3 como base)
modelo = 'facebook/musicgen-melody'
model = musicgen.MusicGen.get_pretrained(modelo, device='cuda')

print("✅ Modelo MELODY cargado con éxito. Ya puedes subir tu archivo 'cancion.mp3'.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


state_dict.bin:   0%|          | 0.00/2.77G [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/audiocraft/models/loaders.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location=device)
Downloadi

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

compression_state_dict.bin:   0%|          | 0.00/236M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/audiocraft/models/loaders.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location=device)
/usr/loca

✅ Modelo MELODY cargado con éxito. Ya puedes subir tu archivo 'cancion.mp3'.


In [ ]:
# Celda 3: Backrooms "Liminal Space" - 3 MINUTOS
import torchaudio
import soundfile as sf
import numpy as np
from google.colab import files
import os
import torch
import gc

# 1. Parámetros de estilo Backrooms
duracion = 180
# Prompt diseñado para ser inquietante, depresivo y nostálgico
prompt = (
    'Backrooms liminal space aesthetic, haunting dark ambient, low-fidelity VHS tape hiss. '
    'SIMPLE: buzzing fluorescent lights sound, distant muffled piano, lonely footsteps. '
    'COMPLEX: detuned nostalgic synthesizers, deep depressive drones, eerie echoes, '
    'slow melancholic orchestral strings, drowning in isolation, pitch-shifted ghosts. '
    'Atmosphere: unsettling, depressive, nostalgic, lonely, 60bpm, cinematic horror.'
)

# Limpieza de VRAM
gc.collect()
torch.cuda.empty_cache()

# Configuración de generación
model.set_generation_params(
    duration=duracion,
    use_sampling=True,
    top_k=250,
    cfg_coef=5.5 # Un poco más alto para resaltar la "depresión" sonora
)

# 2. Ejecución
try:
    if os.path.exists("/content/cancion.mp3"):
        print(f"⏳ Generando Backrooms (3 min) siguiendo tu melodía base...")
        melody_waveform, sr = torchaudio.load("/content/cancion.mp3")
        if melody_waveform.dim() == 2:
            melody_waveform = melody_waveform.unsqueeze(0)

        res = model.generate_with_chroma(
            descriptions=[prompt],
            melody_wavs=melody_waveform,
            melody_sample_rate=sr,
            progress=True
        )
    else:
        print(f"⏳ Generando Backrooms (3 min) solo con texto...")
        res = model.generate([prompt], progress=True)

    # 3. Guardado y Descarga
    print("✅ El vacío ha sido generado. Procesando audio...")
    res_cpu = np.squeeze(res.cpu().numpy())
    ruta_final = '/content/Backrooms_Depressive_Remix.wav'
    sf.write(ruta_final, res_cpu, 32000)

    from audiocraft.utils.notebook import display_audio
    display_audio(res, 32000)
    files.download(ruta_final)

except RuntimeError as e:
    if "out of memory" in str(e).lower():
        print("❌ Error de Memoria: 3 minutos es muy pesado para esta GPU.")
        print("💡 Prueba con 120 segundos.")
    else:
        print(f"❌ Error: {e}")

Output hidden; open in https://colab.research.google.com to view.